In [1]:
# your parameters here...
upstream = ['fetch_n_filter_gdelt_bq', 'normalize_security_names']

In [2]:
# Parameters
upstream = {
    "normalize_security_names": {
        "nb": "/Users/aiujdm2/market_watch/output/notebooks/normalize_security_names.ipynb",
        "data": "/Users/aiujdm2/market_watch/output/data/interim/normalized_security_names.csv",
    },
    "fetch_n_filter_gdelt_bq": {
        "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_n_filter_gdelt_bq.ipynb",
        "data": "/Users/aiujdm2/market_watch/output/data/raw/gdelt_gkg_bqdata-raw.csv",
    },
}
product = {
    "nb": "/Users/aiujdm2/market_watch/output/notebooks/clean_gdelt_data.ipynb",
    "data": "/Users/aiujdm2/market_watch/output/data/interim/gdelt_gkg_data-cleaned.csv",
}


In [3]:
# your code here...
import pandas as pd
import numpy as np
from pathlib import Path
import nltk
import time
import re
from time import time
import json
import ast
from src.utils import preprocess_text
from collections import Counter
from src.utils import fuzz_similarity, preprocess_text
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to /Users/aiujdm2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aiujdm2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/aiujdm2/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/Users/aiujdm2/.local/share/virtualenvs/market_watch-dtlP-L11/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
stops = set(stopwords.words('english'))
pd.options.display.max_colwidth = 200

[nltk_data] Downloading package wordnet to /Users/aiujdm2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aiujdm2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/aiujdm2/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
gdelt_file_path = upstream['fetch_n_filter_gdelt_bq']['data']
securities_file_path = upstream['normalize_security_names']['data']

In [6]:
gdelt_df = pd.read_csv(gdelt_file_path, index_col=0)
securities_df = pd.read_csv(securities_file_path, index_col=0)
securities_df.dropna(subset=['former_name'], inplace=True)
securities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4462 entries, 2 to 9192
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cik          4462 non-null   int64 
 1   ticker       4462 non-null   object
 2   full_name    4462 non-null   object
 3   former_name  4462 non-null   object
dtypes: int64(1), object(3)
memory usage: 174.3+ KB


In [7]:
gdelt_df.columns

Index(['GKGRECORDID', 'DATE', 'SourceCollectionIdentifier',
       'DocumentIdentifier', 'Locations', 'Persons', 'Organizations', 'Tone'],
      dtype='object')

In [8]:
securities_df.head()

,cik,ticker,full_name,former_name
2,1372807,PTMN,portman ridge finance,kcap financial
3,1559444,FURCF,faurecia,faurecia
4,1478320,ADPT,adaptive biotechnologies,adaptive tcr
5,1560905,TRTK,tortec,geo point resources
8,1328598,FXE,invesco currencyshares euro,currencyshares euro


In [9]:
#  drop rows where no organizations names were extracted
gdelt_df = gdelt_df.dropna(subset=['Organizations'])
gdelt_df = gdelt_df.replace(to_replace= ['Google', 'Facebook', 'YouTube', 'Youtube'], value=['alphabet', 'meta platforms', 'alphabet', 'alphabet'], regex=True)

In [10]:
#  get rid of numeric position of org name mention by only extracting alpha names
# Extract only names not the index
gdelt_df.Organizations = gdelt_df.Organizations.map(lambda x: re.split(r',\d+;?', x))

In [11]:
def preprocess_orgs(x):
    return preprocess_text(x, eng=True)

gdelt_df.Organizations = gdelt_df.Organizations.apply(preprocess_orgs)

In [12]:
gdelt_df.Organizations = gdelt_df.Organizations.replace(to_replace=securities_df.former_name.to_list(), value=securities_df.full_name.to_list())

In [13]:
gdelt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11139 entries, 0 to 11173
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   GKGRECORDID                 11139 non-null  object
 1   DATE                        11139 non-null  int64 
 2   SourceCollectionIdentifier  11139 non-null  int64 
 3   DocumentIdentifier          11139 non-null  object
 4   Locations                   9585 non-null   object
 5   Persons                     8992 non-null   object
 6   Organizations               11139 non-null  object
 7   Tone                        11139 non-null  object
dtypes: int64(2), object(6)
memory usage: 783.2+ KB


In [14]:
def count_orgs(x):
    return x if type(x) is float or x is None or len(x) == 0 else Counter(x)

gdelt_df.Organizations = gdelt_df.Organizations.apply(count_orgs)
gdelt_df.dropna(subset=['Organizations'], inplace=True)
gdelt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11139 entries, 0 to 11173
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   GKGRECORDID                 11139 non-null  object
 1   DATE                        11139 non-null  int64 
 2   SourceCollectionIdentifier  11139 non-null  int64 
 3   DocumentIdentifier          11139 non-null  object
 4   Locations                   9585 non-null   object
 5   Persons                     8992 non-null   object
 6   Organizations               11139 non-null  object
 7   Tone                        11139 non-null  object
dtypes: int64(2), object(6)
memory usage: 783.2+ KB


In [15]:
securities_names = preprocess_text(securities_df.full_name)
gdelt_df.Organizations = gdelt_df.Organizations.apply(lambda x: { key:x[key] for key in x.keys() if key in securities_names and key != 'Tooshorttext'})
gdelt_df = gdelt_df[gdelt_df.Organizations.str.len() != 0]

In [16]:
gdelt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9534 entries, 0 to 11173
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   GKGRECORDID                 9534 non-null   object
 1   DATE                        9534 non-null   int64 
 2   SourceCollectionIdentifier  9534 non-null   int64 
 3   DocumentIdentifier          9534 non-null   object
 4   Locations                   8118 non-null   object
 5   Persons                     7953 non-null   object
 6   Organizations               9534 non-null   object
 7   Tone                        9534 non-null   object
dtypes: int64(2), object(6)
memory usage: 670.4+ KB


In [17]:
def split_locations(location_list):
    location_names = []
    if type(location_list) is not float:
        for location_string in location_list:
            loc_parts = location_string.split('#')
            # We are only interested in full location name which is second entry in location string           
            location_names.append(loc_parts[1]) if len(loc_parts) > 1 else np.nan
            
    return location_names
# Locations are semi-colons(;) seperated and each location string is further seperated by hash(#)
gdelt_df.Locations = gdelt_df.Locations.str.split(';').apply(split_locations)
#  Remove duplicates
gdelt_df.Locations = gdelt_df.Locations.map(set)

In [18]:
#  Clean some data elements
gdelt_df.Persons = gdelt_df.Persons.str.findall(pat="[A-Z][a-z]+ [A-Z][a-z]+")
#  Remove duplicates
gdelt_df.Persons = gdelt_df.Persons.map(set, na_action='ignore')

In [19]:
gdelt_df.Tone = gdelt_df.Tone.str.split(',')

In [20]:
# Clean Tone
gdelt_df['AvgTone'] = gdelt_df.Tone.apply(lambda x: x[0])
gdelt_df['PosScore'] = gdelt_df.Tone.apply(lambda x: x[1])
gdelt_df['NegScore'] = gdelt_df.Tone.apply(lambda x: x[2])
gdelt_df['Polarity'] = gdelt_df.Tone.apply(lambda x: x[3])

In [21]:
gdelt_df.drop(["Tone", "DATE", "SourceCollectionIdentifier", "DocumentIdentifier"], axis = 1, inplace=True)

In [22]:
gdelt_df.sample(10)

,GKGRECORDID,Locations,Persons,Organizations,AvgTone,PosScore,NegScore,Polarity
6970,20220329071500-455,"{Berlin, Berlin, Germany, Shanghai, Shanghai, China, India, China}","{Andrej Karpathy, Craig Irwin, Elon Musk, David Trainer}","{'twitter': 1, 'apple': 1, 'tesla': 1}",-0.584795321637427,2.53411306042885,3.11890838206628,5.65302144249513
3067,20220216211500-129,{},"{Bryan Johnson, Elon Musk}",{'tesla': 1},-1.66666666666667,4.33333333333333,6,10.3333333333333
10450,20220308091500-707,"{Kollam, Kerala, India, Chennai, Tamil Nadu, India, India, Nevada, United States, Amravati, Maharashtra, India, Faridabad, Haryana, India, Bengaluru, Karnataka, India, Mysuru, Karnataka, India, Ch...","{Vishwa Vidyapeetham, Thejas Menon, Venkat Rangan, Raghu Raman}",{'tesla': 1},2.19409282700422,3.29113924050633,1.09704641350211,4.38818565400844
6703,20220303060000-604,"{Bengaluru, Karnataka, India, California, United States}","{Bhargav Acharya, Elon Musk, Subhranshu Sahu}",{'tesla': 1},-1.41843971631206,1.41843971631206,2.83687943262411,4.25531914893617
2917,20220411070000-T1073,{United States},NaN,{'twitter': 5},0.321543408360129,1.92926045016077,1.60771704180064,3.53697749196141
9300,20220311171500-1010,"{San Francisco, California, United States, Saudi Arabia, New York, United States, Saudi, California, United States}","{Nicholas Porritt, Edward Chen, Alex Spiro}","{'twitter': 2, 'tesla': 2}",-1.81219110378913,2.47116968698517,4.2833607907743,6.75453047775947
10147,20220324164500-2179,"{Alaska, United States, Argentina, Vaca Muerta, San Juan, Argentina, Wyoming, United States, Colorado, United States, Guyana, Qua Iboe Terminal, Akwa Ibom, Nigeria, Nigeria, North Dakota, United S...","{Craig Thorstenson, Sarah Nordin, Danielle Fugere}","{'devon': 1, 'conocophillips': 1, 'tesla': 1}",-2.0066889632107,1.17056856187291,3.17725752508361,4.34782608695652
1365,20220219233000-49,"{San Francisco, California, United States, New York, United States, London, London, City Of, United Kingdom, Hong Kong, American, Connecticut, United States}","{Julian Robertson, John Lovallo, Andreas Halverson, Andreas Halvorsen}","{'goldman sachs': 1, 'nasdaq': 9, 'chubb': 7}",1.01010101010101,1.72311348781937,0.71301247771836,2.43612596553773
4127,20220415191500-T458,"{Englishman, Canadians, Vancouver, British Columbia, Canada, United States}","{Arabia Alwaleed, Osama Talal, Elon Musk, Gorostiaga Elon}","{'twitter': 17, 'tesla': 1}",0.301204819277108,1.50602409638554,1.20481927710843,2.71084337349398
9319,20220405171500-384,"{Uruguay, China, America, Chinese, Russia, Estonia, Ukraine, Barbados, Bulgaria}","{Vladimir Putin, Mark Zuckerberg, Elon Musk}","{'meta platforms': 1, 'tesla': 1}",-1.14942528735632,1.14942528735632,2.29885057471264,3.44827586206897


In [23]:
output_file_path = product['data']
Path(output_file_path).parent.mkdir(exist_ok=True, parents=True)
gdelt_df.to_csv(output_file_path)
print(f"Saved file {output_file_path}")

Saved file /Users/aiujdm2/market_watch/output/data/interim/gdelt_gkg_data-cleaned.csv


In [24]:
del gdelt_df, securities_df